<a href="https://colab.research.google.com/github/adamfletcherUK/colab_notebooks/blob/main/Keras_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Using Keras in Google Colab

Goal of this notebook is to initialise a simple neural net using the Keras API on colab to test its [colabs] features. 

This is mainly due to my primary computer being an M1 macbook - but I also need to explore some new updates that allow PyTorch to utilise the M1s neural engine.

In [2]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [4]:
train_labels= []
train_samples = []

In [5]:
#example given is a drug given to cohorts of people. People aged <65 had a 5% 
# chance of side effects. While people >65 had a 95% chance of side effects.
# We synthesise the data ourselves as it is quite simplistic

for i in range(50):
  random_younger = randint(13,64)
  train_samples.append(random_younger)
  train_labels.append(1)

  random_older = randint(65, 100)
  train_samples.append(random_older)
  train_labels.append(0)

for i in range(1000):
  random_younger = randint(13,64)
  train_samples.append(random_younger)
  train_labels.append(0)

  random_older = randint(65, 100)
  train_samples.append(random_older)
  train_labels.append(1)

In [31]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels, train_samples = shuffle(train_labels, train_samples)

In [41]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
scaled_train_samples

array([[0.31034483],
       [0.6091954 ],
       [0.4137931 ],
       ...,
       [0.94252874],
       [0.13793103],
       [0.86206897]])

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [34]:
model = Sequential([
            Dense(units=16, input_shape=(1,), activation='relu'),
            Dense(units=32, activation='relu'),
            Dense(units=2, activation='softmax')
])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                32        
                                                                 
 dense_4 (Dense)             (None, 32)                544       
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model.fit(x=scaled_train_samples, 
          y=train_labels, 
          validation_split=0.1,
          batch_size=10, 
          epochs=30,
          shuffle=True)

Epoch 1/30
189/189 [==============================] - 2s 3ms/step - loss: 0.6860 - accuracy: 0.5508 - val_loss: 0.6695 - val_accuracy: 0.6333
Epoch 2/30
189/189 [==============================] - 0s 2ms/step - loss: 0.6531 - accuracy: 0.6566 - val_loss: 0.6414 - val_accuracy: 0.6762
Epoch 3/30
189/189 [==============================] - 1s 8ms/step - loss: 0.6223 - accuracy: 0.7042 - val_loss: 0.6125 - val_accuracy: 0.7048
Epoch 4/30
189/189 [==============================] - 1s 3ms/step - loss: 0.5911 - accuracy: 0.7524 - val_loss: 0.5828 - val_accuracy: 0.7571
Epoch 5/30
189/189 [==============================] - 0s 2ms/step - loss: 0.5592 - accuracy: 0.7868 - val_loss: 0.5532 - val_accuracy: 0.7667
Epoch 6/30
189/189 [==============================] - 1s 3ms/step - loss: 0.5272 - accuracy: 0.8175 - val_loss: 0.5234 - val_accuracy: 0.8048
Epoch 7/30
189/189 [==============================] - 0s 2ms/step - loss: 0.4950 - accuracy: 0.8460 - val_loss: 0.4943 - val_accuracy: 0.8143
Epoch 

In [38]:
test_labels = []
test_samples = []

for i in range(10):
  random_younger = randint(13,64)
  test_samples.append(random_younger)
  test_labels.append(1)

  random_older = randint(65, 100)
  test_samples.append(random_older)
  test_labels.append(0)

for i in range(200):
  random_younger = randint(13,64)
  test_samples.append(random_younger)
  test_labels.append(0)

  random_older = randint(65, 100)
  test_samples.append(random_older)
  test_labels.append(1)

In [39]:
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)
test_labels, test_samples = shuffle(test_labels, test_samples)

In [42]:
scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1,1))

In [43]:
predictions= model.predict(x=scaled_test_samples, batch_size=10, verbose=0)

In [44]:
rounded_predictions = np.argmax(predictions, axis=-1)

In [48]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [49]:
cm = confusion_matrix(y_true=test_labels, y_pred=rounded_predictions)

In [52]:
cm

array([[190,  20],
       [  9, 201]])